In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import spacy
import random
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
from torchtext.data.metrics import bleu_score
import sys
import os
import re
import torchtext

from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
import io
import pandas as pd


from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [2]:
import torch, torchtext
print(torch.__version__)
print(torchtext.__version__)

1.12.1+cu113
0.13.1


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!python -m spacy download zh_core_web_sm
!python -m spacy download en_core_web_sm

2022-10-20 02:07:00.978534: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48.4 MB 2.2 MB/s 
     |████████████████████████████████| 2.4 MB 5.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')
2022-10-20 02:07:17.979014: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:

def getJsonFile(filePath,dir_path = "/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_anki/"):
  return pd.read_json(dir_path+filePath, lines=True, orient='records')

train_json = getJsonFile("anki_train.json")
test_json = getJsonFile("anki_test.json")


total_json = pd.concat([train_json, test_json], axis=0)

zh_tokenizer = get_tokenizer('spacy', language='zh_core_web_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

def tokenize_eng(text):
    text = re.sub(r"([.!?])", r" \1", text.lower())
    return en_tokenizer(text)

def tokenize_zh(text):
    # print(text)
    regex = re.compile(r'[^\u4e00-\u9fa5A-Za-z0-9]')
    text = regex.sub(' ', text.lower())
    return zh_tokenizer(text)


In [6]:

def build_vocab(sentence_list, tokenizer):
  counter = Counter()
  for string_ in sentence_list:
    counter.update(tokenizer(string_))
  return vocab(counter, min_freq =1 , specials=['<unk>', '<bos>', '<eos>', '<pad>'])

# chinese.build_vocab(train_data, max_size=50000, min_freq=50, vectors="glove.6B.100d")
en_vocab = build_vocab(total_json.English, tokenize_eng)
zh_vocab = build_vocab(total_json.Chinese, tokenize_zh)


print ("中文語料的字元表長度: " , len(zh_vocab.vocab) , ", 英文的字元表長度: " ,len(en_vocab.vocab))



中文語料的字元表長度:  14461 , 英文的字元表長度:  6933


In [7]:

def data_process(sentence_list):
  data = []
  for _ , s in sentence_list.iterrows():
    data.append((tokenize_zh(s.Chinese),  tokenize_eng(s.English)))
    # data.append({"zh" : tokenize_zh(s.Chinese), "en" : tokenize_eng(s.English) })
  return data

train_data = data_process(train_json)
test_data = data_process(test_json)

print ("Sample English:", test_data[0][0] , "=> Chinese:", test_data[0][1])

# val_data = data_process(val_filepaths)
# test_data = data_process(test_filepaths)

Sample English: ['我', '最近', '忙', '得', '很'] => Chinese: ['i', "'ve", 'been', 'very', 'busy', 'lately', '.']


In [8]:
print(en_vocab.vocab.get_itos()[1])
en_vocab.vocab.get_stoi()['arabic']

<bos>


4291

In [9]:
def translate_sentence(model, sentence_token, zh_vocab, en_vocab, device, max_length=25):

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    # if type(sentence) == str:
    #     tokens = tokenize_zh(sentence)
    # else:
    #     tokens = [token.lower() for token in sentence]

    # 加入開始符號跟結束符號，代表一個句子
    sentence_token.insert(0, '<bos>')
    sentence_token.append('<eos>')

    # 然後把文字轉自 vactor
    text_to_indices = [zh_vocab.vocab.get_stoi()[token] for token in sentence_token]

    # 再把 vactor list 轉換成 Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # 取消梯度修正
    with torch.no_grad():
        hidden, cell = model.encoder(sentence_tensor)

    # 先宣告 outputs ，然後裡面放一個開符號
    outputs = [en_vocab.vocab.get_stoi()["<bos>"]]


    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        # seq2seq 的decoder 會把上一個 hidden_state 跟 cell 當作input 這是觀念
        # 然後output機率最大的
        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        # 機率最大的數值再把它放進output
        outputs.append(best_guess)

        # 如果是結束字元 eos 的話就中斷不然會一直預測下去
        if output.argmax(1).item() == en_vocab.vocab.get_stoi()["<eos>"]:
            break

    # 再把 vactor 轉成文字， itos = integer to string
    translated_sentence = [en_vocab.vocab.get_itos()[idx] for idx in outputs]

    # remove start token
    return translated_sentence[1:]



def bleu(data, model, tokenize_zh, zh_vocab, en_vocab, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["zh"]
        trg = vars(example)["eng"]

        src_token = tokenize_zh(src)

        prediction = translate_sentence(model, src_token , zh_vocab, en_vocab, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)


def save_checkpoint(state, filename="/content/drive/MyDrive/Colab Notebooks/ithome/checkpoints/seq2seq_cp.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [10]:

class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)

        return hidden, cell


In [11]:
class Decoder(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers, p
    ):
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs)

        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = predictions.squeeze(0)

        return predictions, hidden, cell



In [12]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(en_vocab.vocab)

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        hidden, cell = self.encoder(source)

        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]

        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell = self.decoder(x, hidden, cell)

            # Store next output prediction
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)

            # With probability of teacher_force_ratio we take the actual next word
            # otherwise we take the word that the Decoder predicted it to be.
            # Teacher Forcing is used so that the model gets used to seeing
            # similar inputs at training and testing time, if teacher forcing is 1
            # then inputs at test time might be completely different than what the
            # network is used to. This was a long comment.
            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs

In [13]:
checkPointPath = "/content/drive/MyDrive/Colab Notebooks/ithome/checkpoints/seq2seq_cp.pth.tar"


num_epochs = 100
learning_rate = 0.001
batch_size = 64

# Model hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size_encoder = len(zh_vocab.vocab)
input_size_decoder = len(en_vocab.vocab)
output_size = len(en_vocab.vocab)
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 1024 # Needs to be the same for both RNN's
num_layers = 2
enc_dropout = 0.5
dec_dropout = 0.0

# Tensorboard to get nice loss plot
writer = SummaryWriter(f"/content/drive/MyDrive/Colab Notebooks/ithome/runs/loss_plot")
step = 0


In [14]:

PAD_IDX = zh_vocab.vocab.get_stoi()['<pad>']
BOS_IDX = zh_vocab.vocab.get_stoi()['<bos>']
EOS_IDX = zh_vocab.vocab.get_stoi()['<eos>']


def generate_batch(data_batch):
  zh_batch, en_batch = [], []
  for (zh_token, en_token) in data_batch:
    zh_item = torch.tensor([zh_vocab.vocab.get_stoi()[token] for token in zh_token], dtype=torch.long)
    zh_batch.append(torch.cat([torch.tensor([BOS_IDX]), zh_item, torch.tensor([EOS_IDX])], dim=0))

    en_item = torch.tensor([en_vocab.vocab.get_stoi()[token] for token in en_token], dtype=torch.long)
    en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))

  zh_batch = pad_sequence(zh_batch, padding_value=PAD_IDX)
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return zh_batch, en_batch

train_iter = DataLoader(train_data, batch_size=batch_size,
                        shuffle=True, collate_fn=generate_batch)


test_iter = DataLoader(test_data, batch_size=batch_size,
                        shuffle=True, collate_fn=generate_batch)


In [15]:

encoder_net = Encoder(input_size_encoder, encoder_embedding_size, hidden_size, num_layers, enc_dropout).to(device)

decoder_net = Decoder(
    input_size_decoder,
    decoder_embedding_size,
    hidden_size,
    output_size,
    num_layers,
    dec_dropout,
).to(device)



In [17]:

model = Seq2Seq(encoder_net, decoder_net).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = en_vocab.vocab.get_stoi()["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

if os.path.isfile(checkPointPath):
    load_checkpoint(torch.load(checkPointPath), model, optimizer)

=> Loading checkpoint


RuntimeError: ignored

In [ ]:

sentence = "你想不想要来我家看猫?或者一起看电影?"
example_token = tokenize_zh(sentence)

for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")

    checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
    save_checkpoint(checkpoint)

    model.eval()

    translated_sentence = translate_sentence(
        model, example_token, zh_vocab, en_vocab, device, max_length=50
    )

    print(f"Translated example sentence: \n {translated_sentence}")

    model.train()

    for batch_idx, batch in enumerate(train_iter):
        # Get input and targets and get to cuda
        inp_data = batch[0].to(device)
        target = batch[1].to(device)

        # Forward prop
        output = model(inp_data, target)
        # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
        # doesn't take input in that form. For example if we have MNIST we want to have
        # output to be: (N, 10) and targets just (N). Here we can view it in a similar
        # way that we have output_words * batch_size that we want to send in into
        # our cost function, so we need to do some reshapin. While we're at it
        # Let's also remove the start token while we're at it
        output = output[1:].reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output, target)

        # Back prop
        loss.backward()

        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()

        # Plot to tensorboard
        writer.add_scalar("Training loss", loss, global_step=step)
        step += 1


score = bleu(test_data[1:100], model, chinese, english, device)
print(f"Bleu score {score*100:.2f}")